In [9]:
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor

ModuleNotFoundError: No module named 'autogluon'

In [7]:
#read excel file
#dataset = pandas.read_csv('Dataset.csv')
#get label dataframe, usually last column
# #y = df.iloc[:,-1:]
#get all features
# #X = df. loc[:, df. columns != 'labels']
#placeholder features and labels
X = np.zeros((10,6))
y = np.ones((10,1))

In [8]:
#create training set, validation set and test set 0.8:0.1:0.1 division
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size =0.5, random_state=1)

In [ ]:
#join training features and labels for autogluon
train_set = pd.concat([X_train, y_train], axis=1)
#get label column name
label = 'label'
#run algorithms on training set
predictor = TabularPredictor(label=label).fit(train_set)
val_set =  pd.concat([X_val, y_val], axis=1)
predictor.leaderboard(val_set, silent=True)